In [1]:
import pandas as pd
import sqlite3

In [3]:
cnx = sqlite3.connect('foo.db')

In [5]:
df_customers = pd.read_sql_query("SELECT * FROM olist_customers_dataset", cnx)
df_geolocations = pd.read_sql_query("SELECT * FROM olist_geolocation_dataset", cnx)
df_orderitems = pd.read_sql_query("SELECT * FROM olist_order_items_dataset", cnx)
df_payments = pd.read_sql_query("SELECT * FROM olist_order_payments_dataset", cnx)
df_reviews = pd.read_sql_query("SELECT * FROM olist_order_reviews_dataset", cnx)
df_orders = pd.read_sql_query("SELECT * FROM olist_orders_dataset", cnx)
df_products = pd.read_sql_query("SELECT * FROM olist_products_dataset", cnx)
df_sellers = pd.read_sql_query("SELECT * FROM olist_sellers_dataset", cnx)

In [57]:
import pandas as pd

# Misalkan df_customers dan df_orders sudah didefinisikan sebelumnya
# df_customers = pd.read_csv('path_to_customers.csv')
# df_orders = pd.read_csv('path_to_orders.csv')

# Langkah 1: Mengonversi kolom order_purchase_timestamp ke format tanggal YYYY-MM-DD
df_orders['order_purchase_timestamp'] = pd.to_datetime(df_orders['order_purchase_timestamp']).dt.strftime('%Y-%m-%d')

# Langkah 2: Melakukan merge antara df_orders dan df_customers
merged_df = pd.merge(df_orders, df_customers[['customer_id', 'customer_unique_id']], on='customer_id', how='left')

# Langkah 3: Mengonversi kembali order_purchase_timestamp ke tipe datetime untuk pengelompokan
merged_df['order_purchase_timestamp'] = pd.to_datetime(merged_df['order_purchase_timestamp'])

# Langkah 4: Mengelompokkan data berdasarkan customer_unique_id dan mendapatkan tanggal transaksi awal dan akhir
df_grouped = merged_df.groupby('customer_unique_id')['order_purchase_timestamp'].agg(['min', 'max']).reset_index()

# Langkah 5: Menghitung selisih hari antara transaksi awal dan akhir
df_grouped['days_difference'] = (df_grouped['max'] - df_grouped['min']).dt.days

# Mengganti nama kolom untuk kejelasan
df_grouped.columns = ['customer_unique_id', 'first_transaction_date', 'last_transaction_date', 'days_difference']

# Menampilkan hasil
df_grouped


,customer_unique_id,first_transaction_date,last_transaction_date,days_difference
0,0000366f3b9a7992bf8c76cfdf3221e2,2018-05-10,2018-05-10,0
1,0000b849f77a49e4a4ce2b2a4ca5be3f,2018-05-07,2018-05-07,0
2,0000f46a3911fa3c0805444483337064,2017-03-10,2017-03-10,0
3,0000f6ccb0745a6a4b88665a16c9f078,2017-10-12,2017-10-12,0
4,0004aac84e0df4da2b147fca70cf8255,2017-11-14,2017-11-14,0
...,...,...,...,...
96091,fffcf5a5ff07b0908bd4e2dbc735a684,2017-06-08,2017-06-08,0
96092,fffea47cd6d3cc0a88bd621562a9d061,2017-12-10,2017-12-10,0
96093,ffff371b4d645b6ecea244b27531430a,2017-02-07,2017-02-07,0
96094,ffff5962728ec6157033ef9805bacc48,2018-05-02,2018-05-02,0


In [65]:
df_grouped.value_counts('days_difference')

days_difference
0      93947
1         46
2         38
6         30
3         27
       ...  
335        1
332        1
330        1
328        1
633        1
Name: count, Length: 428, dtype: int64

In [67]:
df_grouped[df_grouped['days_difference'] == 633]

,customer_unique_id,first_transaction_date,last_transaction_date,days_difference
19160,32ea3bdedab835c3aa6cb68ce66565ef,2016-10-03,2018-06-28,633


In [47]:
import pandas as pd

# Misalkan df_customers dan df_orders sudah didefinisikan sebelumnya
# df_customers = pd.read_csv('path_to_customers.csv')
# df_orders = pd.read_csv('path_to_orders.csv')

# Mengonversi kolom order_purchase_timestamp ke tipe datetime
df_orders['order_purchase_timestamp'] = pd.to_datetime(df_orders['order_purchase_timestamp'])

# Mengelompokkan data berdasarkan customer_id dan mendapatkan tanggal transaksi awal dan akhir
df_grouped = df_orders.groupby('customer_id')['order_purchase_timestamp'].agg(['min', 'max']).reset_index()

# Menghitung selisih hari antara transaksi awal dan akhir
df_grouped['days_difference'] = (df_grouped['max'] - df_grouped['min']).dt.days

# Menggabungkan dengan df_customers untuk mendapatkan customer_unique_id
result_df = pd.merge(df_customers[['customer_id', 'customer_unique_id']], df_grouped, on='customer_id', how='left')

# Mengganti nama kolom untuk kejelasan
result_df.columns = ['customer_id', 'customer_unique_id', 'first_transaction_date', 'last_transaction_date', 'days_difference']

# Menampilkan hasil
result_df.sort_values('days_difference')


,customer_id,customer_unique_id,first_transaction_date,last_transaction_date,days_difference
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,2017-05-16 15:05:35,2017-05-16 15:05:35,0
1,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,2018-01-12 20:48:24,2018-01-12 20:48:24,0
2,4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,2018-05-19 16:07:45,2018-05-19 16:07:45,0
3,b2b6027bc5c5109e529d4dc6358b12c3,259dac757896d24d7702b9acbbff3f3c,2018-03-13 16:06:38,2018-03-13 16:06:38,0
4,4f2d8ab171c80ec8364f7c12e35b23ad,345ecd01c38d18a9036ed96c73b8d066,2018-07-29 09:51:30,2018-07-29 09:51:30,0
...,...,...,...,...,...
99436,17ddf5dd5d51696bb3d7c6291687be6f,1a29b476fee25c95fbafc67c5ac95cf8,2018-04-07 15:48:17,2018-04-07 15:48:17,0
99437,e7b71a9017aa05c9a7fd292d714858e8,d52a67c98be1cf6a5c84435bd38d095d,2018-04-04 08:20:22,2018-04-04 08:20:22,0
99438,5e28dfe12db7fb50a4b2f691faecea5e,e9f50caf99f032f0bf3c55141f019d99,2018-04-08 20:11:50,2018-04-08 20:11:50,0
99439,56b18e2166679b8a959d72dd06da27f9,73c2643a0a458b49f58cea58833b192e,2017-11-03 21:08:33,2017-11-03 21:08:33,0


In [61]:
result_df.sort_values('days_difference')

,customer_id,customer_unique_id,first_transaction_date,last_transaction_date,days_difference
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,2017-05-16 15:05:35,2017-05-16 15:05:35,0
66299,229ac14c6ee6d6a606264ebcc861beb7,493f3e2114badbd36273531549b6d21c,2018-08-08 20:25:21,2018-08-08 20:25:21,0
66298,8cd118ecfb9703463ef9670b1d18a26e,c6e994767dccb6b8a8207c1f44451159,2017-10-17 11:12:57,2017-10-17 11:12:57,0
66297,8b7369c82ac3d8d71b77d5b23b085696,98de6fdee8013609f315bcbcf39fe23f,2018-05-13 17:08:57,2018-05-13 17:08:57,0
66296,4b7decb9b58e2569548b8b4c8e20e8d7,ff22e30958c13ffe219db7d711e8f564,2018-09-03 09:06:57,2018-09-03 09:06:57,0
...,...,...,...,...,...
33141,6b05c8f3a25a71005d5eb9b51766c333,1ead5f5c37bde289f52f322c4c497c4b,2017-08-15 20:11:01,2017-08-15 20:11:01,0
33140,afddd76d1457bf3429973d37105f7aef,b51e5231c8ece4e6f463f2add2e07f60,2018-08-23 11:01:11,2018-08-23 11:01:11,0
33139,7a8a02e54cf3472df427e927f8b9c046,9c3af16efacb7aa06aa3bc674556c5d6,2018-09-25 11:59:18,2018-09-25 11:59:18,0
33161,80fd8c4c7004d98ff9cdf61587724a35,3d6042d877fddc23b9d57ed6669a5b8c,2017-07-07 13:29:30,2017-07-07 13:29:30,0


In [49]:
df_orders.groupby('customer_unique_id')

KeyError: 'customer_unique_id'

In [31]:
result_

,customer_id,customer_unique_id,first_transaction_date,last_transaction_date,days_difference
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,2017-05-16 15:05:35,2017-05-16 15:05:35,0
1,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,2018-01-12 20:48:24,2018-01-12 20:48:24,0
2,4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,2018-05-19 16:07:45,2018-05-19 16:07:45,0
3,b2b6027bc5c5109e529d4dc6358b12c3,259dac757896d24d7702b9acbbff3f3c,2018-03-13 16:06:38,2018-03-13 16:06:38,0
4,4f2d8ab171c80ec8364f7c12e35b23ad,345ecd01c38d18a9036ed96c73b8d066,2018-07-29 09:51:30,2018-07-29 09:51:30,0
...,...,...,...,...,...
99436,17ddf5dd5d51696bb3d7c6291687be6f,1a29b476fee25c95fbafc67c5ac95cf8,2018-04-07 15:48:17,2018-04-07 15:48:17,0
99437,e7b71a9017aa05c9a7fd292d714858e8,d52a67c98be1cf6a5c84435bd38d095d,2018-04-04 08:20:22,2018-04-04 08:20:22,0
99438,5e28dfe12db7fb50a4b2f691faecea5e,e9f50caf99f032f0bf3c55141f019d99,2018-04-08 20:11:50,2018-04-08 20:11:50,0
99439,56b18e2166679b8a959d72dd06da27f9,73c2643a0a458b49f58cea58833b192e,2017-11-03 21:08:33,2017-11-03 21:08:33,0


In [29]:
import pandas as pd

# Misalkan df_customers dan df_orders sudah didefinisikan sebelumnya
# df_customers = pd.read_csv('path_to_customers.csv')
# df_orders = pd.read_csv('path_to_orders.csv')

# Mengonversi kolom order_purchase_timestamp ke tipe datetime
df_orders['order_purchase_timestamp'] = pd.to_datetime(df_orders['order_purchase_timestamp'])

# Mengelompokkan data berdasarkan customer_id dan mendapatkan tanggal transaksi awal dan akhir
df_grouped = df_orders.groupby('customer_id')['order_purchase_timestamp'].agg(['min', 'max']).reset_index()

# Menghitung selisih hari antara transaksi awal dan akhir
df_grouped['days_difference'] = (df_grouped['max'] - df_grouped['min']).dt.days

# Menggabungkan dengan df_customers untuk mendapatkan customer_id
result_df = pd.merge(df_customers[['customer_id']], df_grouped, on='customer_id', how='left')

# Mengganti nama kolom untuk kejelasan
result_df.columns = ['customer_id', 'first_transaction_date', 'last_transaction_date', 'days_difference']

# Menampilkan hasil
print(result_df)


                            customer_id first_transaction_date  \
0      06b8999e2fba1a1fbc88172c00ba8bc7    2017-05-16 15:05:35   
1      18955e83d337fd6b2def6b18a428ac77    2018-01-12 20:48:24   
2      4e7b3e00288586ebd08712fdd0374a03    2018-05-19 16:07:45   
3      b2b6027bc5c5109e529d4dc6358b12c3    2018-03-13 16:06:38   
4      4f2d8ab171c80ec8364f7c12e35b23ad    2018-07-29 09:51:30   
...                                 ...                    ...   
99436  17ddf5dd5d51696bb3d7c6291687be6f    2018-04-07 15:48:17   
99437  e7b71a9017aa05c9a7fd292d714858e8    2018-04-04 08:20:22   
99438  5e28dfe12db7fb50a4b2f691faecea5e    2018-04-08 20:11:50   
99439  56b18e2166679b8a959d72dd06da27f9    2017-11-03 21:08:33   
99440  274fa6071e5e17fe303b9748641082c8    2017-12-19 14:27:23   

      last_transaction_date  days_difference  
0       2017-05-16 15:05:35                0  
1       2018-01-12 20:48:24                0  
2       2018-05-19 16:07:45                0  
3       2018-03-13 

In [21]:
df_customers.shape

(99441, 5)

In [23]:
df_orders.shape

(99441, 8)

In [17]:
df_grouped.sort_values('days_difference')

,customer_id,min,max,days_difference
0,00012a2ce6f8dcda20d059ce98491703,2017-11-14 16:08:26,2017-11-14 16:08:26,0
66299,aa72002446dfbac0691304465c1b982d,2017-02-12 12:32:38,2017-02-12 12:32:38,0
66298,aa71b21388b99444414a772acde9f6aa,2017-09-18 13:58:03,2017-09-18 13:58:03,0
66297,aa6ee75a5950e07fa3f543bf48ee608e,2018-05-07 14:52:26,2018-05-07 14:52:26,0
66296,aa6edca5d83d4c8e9a2262b9847febd6,2018-08-01 14:58:41,2018-08-01 14:58:41,0
...,...,...,...,...
33141,558de2f6863bf7588338479d614522a3,2017-10-25 16:33:09,2017-10-25 16:33:09,0
33140,558db91121d704e92ef88de10aa722fd,2018-04-27 20:21:17,2018-04-27 20:21:17,0
33139,558db5ecbdc2c47fa8b3f8ae3b8a0453,2018-02-07 10:06:39,2018-02-07 10:06:39,0
33161,559f9d302aa523b2dd718e956f248db5,2018-05-27 17:59:00,2018-05-27 17:59:00,0


In [13]:
result_df.sort_values('days_difference')

,customer_id,first_transaction_date,last_transaction_date,days_difference
0,06b8999e2fba1a1fbc88172c00ba8bc7,2017-05-16 15:05:35,2017-05-16 15:05:35,0
66299,229ac14c6ee6d6a606264ebcc861beb7,2018-08-08 20:25:21,2018-08-08 20:25:21,0
66298,8cd118ecfb9703463ef9670b1d18a26e,2017-10-17 11:12:57,2017-10-17 11:12:57,0
66297,8b7369c82ac3d8d71b77d5b23b085696,2018-05-13 17:08:57,2018-05-13 17:08:57,0
66296,4b7decb9b58e2569548b8b4c8e20e8d7,2018-09-03 09:06:57,2018-09-03 09:06:57,0
...,...,...,...,...
33141,6b05c8f3a25a71005d5eb9b51766c333,2017-08-15 20:11:01,2017-08-15 20:11:01,0
33140,afddd76d1457bf3429973d37105f7aef,2018-08-23 11:01:11,2018-08-23 11:01:11,0
33139,7a8a02e54cf3472df427e927f8b9c046,2018-09-25 11:59:18,2018-09-25 11:59:18,0
33161,80fd8c4c7004d98ff9cdf61587724a35,2017-07-07 13:29:30,2017-07-07 13:29:30,0


In [ ]:
df_orderitems.query('order_id == "f80549a97eb203e1566e026ab66f045b"')

In [ ]:
df_payments

In [ ]:
df_orderitems.sort_values('order_id')

In [ ]:
df_orders.head()

In [ ]:
df_customers.head()

In [ ]:
df_orderitems.shape

In [ ]:
df_orderitems.nunique()

In [ ]:
df_orders.sort_values('order_id')

In [ ]:
df_customers.to_csv('df_customers.csv', index=False)

In [ ]:
df_geolocations.to_csv('df_geolocations.csv', index=False)

In [ ]:
df_orderitems.to_csv('df_orderitems.csv', index=False)

In [ ]:
df_payments.to_csv('df_payments.csv', index=False)

In [ ]:
df_reviews.to_csv('df_reviews.csv', index=False)

In [ ]:
df_orders.to_csv('df_orders.csv', index=False)

In [ ]:
df_products.to_csv('df_products.csv', index=False)

In [ ]:
df_sellers.to_csv('df_sellers.csv', index=False)

In [ ]:
print("1")
df_customers.shape

In [ ]:
print("2")
df_geolocations.shape

In [ ]:
print("3")
df_orderitems.shape

In [ ]:
print("4")
df_payments.shape

In [ ]:
print("5")
df_reviews.shape

In [ ]:
print("6")
df_orders.shape

In [ ]:
print("7")
df_products.shape

In [ ]:
print("8")
df_sellers.shape

In [ ]:
null_df_customer = df_customers.isnull().sum().sum()
null_df_geolocations = df_geolocations.isnull().sum().sum()
null_df_orderitems = df_orderitems.isnull().sum().sum()
null_df_payments = df_payments.isnull().sum().sum()
null_df_reviews = df_reviews.isnull().sum().sum()
null_df_orders = df_orders.isnull().sum().sum()
null_df_products = df_products.isnull().sum().sum()
null_df_sellers = df_sellers.isnull().sum().sum()

In [ ]:
print('Number of NaN values null_df_customer:', null_df_customer)
print('Number of NaN values null_df_geolocations:', null_df_geolocations)
print('Number of NaN values null_df_orderitems:', null_df_orderitems)
print('Number of NaN values null_df_payments:', null_df_payments)
print('Number of NaN values null_df_reviews:', null_df_reviews)
print('Number of NaN values null_df_orders:', null_df_orders)
print('Number of NaN values null_df_products:', null_df_products)
print('Number of NaN values null_df_sellers:', null_df_sellers)


In [ ]:
# Menggabungkan DataFrame
# Pertama, kita gabungkan df_orders dengan df_customers
merged_df = pd.merge(df_orders, df_customers, on='customer_id', how='inner')

# Menggabungkan dengan df_orderitems
merged_df = pd.merge(merged_df, df_orderitems, on='order_id', how='inner')

# Menggabungkan dengan df_payments
merged_df = pd.merge(merged_df, df_payments, on='order_id', how='inner')

# Menggabungkan dengan df_reviews
merged_df = pd.merge(merged_df, df_reviews, on='order_id', how='inner')

# Menggabungkan dengan df_products
merged_df = pd.merge(merged_df, df_products, on='product_id', how='inner')

# Menggabungkan dengan df_sellers
merged_df = pd.merge(merged_df, df_sellers, on='seller_id', how='inner')

# Menggabungkan dengan df_geolocations
merged_df = pd.merge(merged_df, df_geolocations, left_on='customer_zip_code_prefix', right_on='geolocation_zip_code_prefix', how='inner')


In [ ]:
merged_df.isna().sum()

In [ ]:
merged_df.shape

In [ ]:
# Menggabungkan df_orders dengan df_orderitems berdasarkan order_id
df = pd.merge(df_orderitems, df_orders, on='order_id', how='left')

# Menggabungkan dengan df_payments berdasarkan order_id
df = pd.merge(df, df_payments, on='order_id', how='left')

# Menggabungkan dengan df_reviews berdasarkan order_id
df = pd.merge(df, df_reviews, on='order_id', how='left')

# Menggabungkan dengan df_products berdasarkan product_id
df = pd.merge(df, df_products, on='product_id', how='left')

# Menggabungkan dengan df_sellers berdasarkan seller_id
df = pd.merge(df, df_sellers, on='seller_id', how='left')

# Menggabungkan dengan df_customers berdasarkan customer_id
df = pd.merge(df, df_customers, on='customer_id', how='left')

# Menggabungkan dengan df_geolocations berdasarkan zip_code_prefix
df = pd.merge(df, df_geolocations, left_on='customer_zip_code_prefix', right_on='geolocation_zip_code_prefix', how='left')

In [ ]:
df.shape

In [ ]:
df.isna().sum()

In [ ]:
df = pd.merge(df, df_geolocations, left_on='customer_zip_code_prefix', right_on='geolocation_zip_code_prefix', how='left')

In [ ]:
# Menggabungkan dengan df_customers berdasarkan customer_id
df = pd.merge(df, df_customers, on='customer_id', how='left')

In [ ]:
df = pd.merge(df, df_orderitems, on='order_id', how='left')

In [ ]:
# Menggabungkan dengan df_reviews berdasarkan order_id
df = pd.merge(df_orders, df_reviews, on='order_id', how='left')

In [ ]:
# Menggabungkan dengan df_payments berdasarkan order_id
df = pd.merge(df, df_payments, on='order_id', how='left')

In [ ]:
# Menggabungkan dengan df_products berdasarkan product_id
df = pd.merge(df, df_products, on='product_id', how='left')

In [ ]:
# Menggabungkan dengan df_sellers berdasarkan seller_id
df = pd.merge(df, df_sellers, on='seller_id', how='left')

In [ ]:
df.isna().sum()

In [ ]:
df_combined[df_combined['customer_state'].astype(str).str.contains('DF')]

In [ ]:
df_combined

In [ ]:
#Check null Values dataframes
df_combined.isna().sum()

In [ ]:
df_null_geo = df_combined[df_combined['geolocation_city'].isna()]

In [ ]:
df_null_geo.isna().sum()

In [ ]:
df_null_pay =  df_combined[df_combined['payment_value'].isna()]

In [ ]:
df_null_pay.isna().sum()

In [ ]:
df_null_pay

In [ ]:
df_null_pay[['order_id', 'product_id', 'price', 'order_status']].value_counts('order_status')

In [ ]:
df_null_review = df_combined[df_combined['review_id'].isna()]

In [ ]:
df_nulls_drop = df_combined.dropna(subset=['review_id', 'review_score', 'review_comment_title', 'review_comment_message', 'review_creation_date', 'review_answer_timestamp'])

In [ ]:
df_nulls_drop.value_counts('order_status')

In [ ]:
df_nulls_drop.shape

In [ ]:
df_null_review.value_counts('order_status')

In [ ]:
df_null_pay

In [ ]:
# Misalkan df_combined adalah DataFrame yang sudah digabungkan
# Group by 'order_status' dan hitung jumlah nilai null di 'review_score'
null_counts = df_combined.groupby('order_status')['review_score'].apply(lambda x: x.isnull().sum())

# Tampilkan hasil
null_counts


In [ ]:
df_clean = df.copy()

In [ ]:
df_clean['payment_installments'] = pd.to_numeric(df_clean['payment_installments'], errors='coerce')
df_clean['payment_value'] = pd.to_numeric(df_clean['payment_value'], errors='coerce')

#Cleansing null_values untuk data payment
df_clean['payment_installments'] = df_clean['payment_installments'].interpolate()
df_clean['payment_value'] = df_clean['payment_value'].interpolate()

In [ ]:
df_clean.isna().sum()

In [ ]:
mode_payment_type = df_clean['payment_type'].mode()[0]
df_clean['payment_type'].fillna(mode_payment_type, inplace=True)

In [ ]:
df_clean['payment_sequential'].fillna(method='ffill', inplace=True)